In [1]:
from Common import *
from models import *
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from logger import *    
from peewee import *

s=Service(ChromeDriverManager().install())

logger = get_logger(path = 'test_sdmx.log')




====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\riccie\.wdm\drivers\chromedriver\win32\97.0.4692.71]


In [2]:
logger.info('**** START ****')
driver = webdriver.Chrome(service=s)
url = 'https://www.istat.it/it/metodi-e-strumenti/web-service-sdmx#Aggiornamentodeidataseterelativequery-2'
driver.get(url )

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "rsssep"))
    )
    page_source = driver.page_source

finally:
    driver.quit()


[2022-01-18 08:24:52,426]:[sdmx_istat]:[<module>]:[INFO]: **** START ****


In [3]:
from bs4 import BeautifulSoup
import lxml
soup = BeautifulSoup(page_source,'html.parser')


In [4]:
#Common
def Get_box_info(box):

    ######################## REGEX ##############################
    import re
    start_regex=re.compile('(?<=<query:StartTime>)(.*)(?=</query:StartTime>)')
    end_regex=re.compile('(?<=<query:EndTime>)(.*)(?=</query:EndTime>)')
    id_regex=re.compile('(?<=<ID>)(.*)(?=</ID>)')
    ##############################################################
    
    titolo=box.find(class_='title').text
    
    data=box.find(class_='date').text
    
    text=box.find('textarea').text
    
    start=start_regex.findall(text)[0]
    end=end_regex.findall(text)[0]
    
    ID=id_regex.findall(text)[0]
    
    return dict(titolo=titolo,
                data_aggiornamento=data,
                start_periodo=start,
                end_periodo=end,
                flow=ID)



In [5]:
### aggiunta gestione casistica unica pagina di aggiornamenti
try:

    l=soup.find(class_='simplePagerNav')
    result=[]
    ndt=0
    len(l)
    logger.debug('Casistica multi pagina')
    info_list=[]
    for i in list(range(1, len(l)+1)):
        cl='simplePagerPage'+str(i)
        page=soup.find_all(class_=cl)
        for box in page:
                    print('______________ box numero', i, '_______________')
                    Info=Get_box_info(box)
                    print(Info['titolo'])
                    print('aggiornato il ' + Info['data_aggiornamento'])
                    print('info per query -> ',  Info['flow'],  Info['start_periodo'], Info['end_periodo'])
                    print(' ')
                    i=i+1
                    ndt=ndt+1 
                    info_list.append(Info)
    logger.info(f'Identificate {len(l)} pagine di aggiornamenti per un totale di {ndt} dataset')
    
except TypeError as e :   
    logger.error(e)
    logger.debug('Except - casistica unica pagina')
    cl='simplePagerPage1'
    page=soup.find_all(class_=cl)
    i=1
    info_list=[]
    for box in page:
        print('______________ box numero', i, '_______________')
        Info=Get_box_info(box)
        print(Info['titolo'])
        print('aggiornato il ' + Info['data_aggiornamento'])
        print('info per query -> ',  Info['flow'], '-' ,Info['start_periodo'], '-' ,Info['end_periodo'])
        print(' ')
        i=i+1
        info_list.append(Info)

    logger.info(f'Identificata 1 pagina di aggiornamenti per un totale di {len(page)} dataset ')
      



[2022-01-18 08:24:59,783]:[sdmx_istat]:[<module>]:[INFO]: Identificate 2 pagine di aggiornamenti per un totale di 13 dataset


______________ box numero 1 _______________
Ipca - medie annue dal 2001 (base 2015)
aggiornato il 17/01/2022 11:30
info per query ->  168_758 2020 2021
 
______________ box numero 2 _______________
IPCA a tassazione costante - Medie annue dal 2002 (base 2015=100)
aggiornato il 17/01/2022 10:29
info per query ->  168_757 2021 2021
 
______________ box numero 3 _______________
Interruzioni volontarie della gravidanza - Caratteristiche della donna
aggiornato il 17/01/2022 01:58
info per query ->  42_70 2020 2020
 
______________ box numero 4 _______________
Foi - medie annue dal 2016 (base 2015)
aggiornato il 17/01/2022 11:58
info per query ->  169_746 2021 2021
 
______________ box numero 5 _______________
FOI - Dati mensili dal 2016
aggiornato il 17/01/2022 10:11
info per query ->  169_745 2021 2021
 
______________ box numero 6 _______________
Ipca - mensili dal 2001 (base 2015)
aggiornato il 17/01/2022 10:18
info per query ->  168_760 2021 2021
 
______________ box numero 7 __________

In [6]:

######### Scrivo in db i risultati dello scraping
    ######### aggiunto controllo per risultati già scritti

   
n=0
for row in info_list:

    try:

        n=n+1
        flow=row['flow']
        titolo=row['titolo']
        data_aggiornamento=row['data_aggiornamento']
        cur=db.cursor()
        cur.execute(f'select count(*) from monitor where flow="{flow}" and data_aggiornamento="{data_aggiornamento}"')
        res=cur.fetchall()[0][0]

        if res==0:
            
            logger.info(f'inserisco riga {n} - {flow} in db')

            Monitor.create(**row)
        
        else:

            logger.info(f'Aggiornamento flow {flow} già presente in db')
            
    except Exception as e:
    
        logger.error(e)

db.close()

[2022-01-18 08:25:11,429]:[sdmx_istat]:[<module>]:[INFO]: Aggiornamento flow 168_758 già presente in db
[2022-01-18 08:25:11,430]:[sdmx_istat]:[<module>]:[INFO]: Aggiornamento flow 168_757 già presente in db
[2022-01-18 08:25:11,431]:[sdmx_istat]:[<module>]:[INFO]: Aggiornamento flow 42_70 già presente in db
[2022-01-18 08:25:11,432]:[sdmx_istat]:[<module>]:[INFO]: Aggiornamento flow 169_746 già presente in db
[2022-01-18 08:25:11,433]:[sdmx_istat]:[<module>]:[INFO]: Aggiornamento flow 169_745 già presente in db
[2022-01-18 08:25:11,434]:[sdmx_istat]:[<module>]:[INFO]: Aggiornamento flow 168_760 già presente in db
[2022-01-18 08:25:11,435]:[sdmx_istat]:[<module>]:[INFO]: inserisco riga 7 - 121_331 in db
[2022-01-18 08:25:11,446]:[sdmx_istat]:[<module>]:[INFO]: Aggiornamento flow 168_756 già presente in db
[2022-01-18 08:25:11,447]:[sdmx_istat]:[<module>]:[INFO]: Aggiornamento flow 167_744 già presente in db
[2022-01-18 08:25:11,448]:[sdmx_istat]:[<module>]:[INFO]: Aggiornamento flow 16

True

In [19]:
rows=Monitor.select()

for row in rows:
    print(f' Il data-set {row.titolo} è stato aggiornato il {row.data_aggiornamento.split()[0]} alle {row.data_aggiornamento.split()[1]}, download completato: {row.download}')


 Il data-set Indicatori  permessi di costruire è stato aggiornato il 14/01/2022 alle 10:12, download completato: False
 Il data-set Nic - mensili dal 2016 (base 2015) è stato aggiornato il 17/01/2022 alle 10:10, download completato: False
 Il data-set IPCA a tassazione costante - Dati mensili dal 2002 (base 2015=100) è stato aggiornato il 17/01/2022 alle 10:21, download completato: False
 Il data-set Ipca - medie annue dal 2001 (base 2015) è stato aggiornato il 17/01/2022 alle 10:23, download completato: False
 Il data-set IPCA a tassazione costante - Medie annue dal 2002 (base 2015=100) è stato aggiornato il 17/01/2022 alle 10:29, download completato: False
 Il data-set FOI - Dati mensili dal 2016 è stato aggiornato il 17/01/2022 alle 10:11, download completato: False
 Il data-set Ipca - mensili dal 2001 (base 2015) è stato aggiornato il 17/01/2022 alle 10:18, download completato: False
 Il data-set Indicatori  permessi di costruire è stato aggiornato il 14/01/2022 alle 10:12, downloa

In [8]:
rows=Monitor.select()

for row in rows:
    print(f'  {row.titolo}, download completato: {row.download} {row.volume},{row.note} , ')


In [2]:
from Common import *
from peewee import *
from logger import *
from models import *  
import xml.etree.ElementTree as ET
import subprocess
import os.path
import time
######### leggo i risultati dello scraping e seleziono le nuove osservazioni da scaricare
    ######### Aggiunto scarico + parsing delle constraint + individuazione livello geografico 
        ######### aggiunta modifica dei campi Download e Volume post scarico
            ###### gestito parametro frequenza in base alle constraint(si provano tutte)
                ######## TODO: gestire scarichi multipli in base ai livelli geografici minimi disponibili
                                # Al momento scarica al livello geo minimo disponibile
                    ###### Aggiunto controllo sulla pre-esistenza del file constraint.xml

   

cur=db.cursor()
cur.execute('select * from monitor where download=False')
result=cur.fetchall()
db.close()
for flow in result:
    
    flowRef=flow[3]
    start=flow[5]
    end=flow[6]
    data_agg=flow[4]


In [6]:
anni=range(start,end)
for a in anni:
    print(a)

2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
